In [6]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV, KFold
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

print("1. Veri Setini Yükleme")
# 1. Veri Setini Yükleme
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')

print("2. Veri Ön İşleme ve Temizleme")

# a. Yaş hesaplama ve 'Yas' sütununu ekleme
print(" - Yaş hesaplanıyor")

def calculate_age(born, application_date):
    try:
        born = pd.to_datetime(born, errors='coerce', dayfirst=True)
        if pd.isnull(born):
            return np.nan
        age = (application_date - born).days / 365.25
        return age
    except:
        return np.nan

# Başvuru tarihini oluşturun (varsayım: başvuru tarihi yılın ortası)
train_df['Basvuru Tarihi'] = pd.to_datetime(train_df['Basvuru Yili'], format='%Y') + pd.DateOffset(months=6)
test_df['Basvuru Tarihi'] = pd.to_datetime(test_df['Basvuru Yili'], format='%Y') + pd.DateOffset(months=6)

train_df['Yas'] = train_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Tarihi']), axis=1)
test_df['Yas'] = test_df.apply(lambda row: calculate_age(row['Dogum Tarihi'], row['Basvuru Tarihi']), axis=1)

# Eksik veya mantıksız yaşları kontrol edelim
for df in [train_df, test_df]:
    median_age = df['Yas'].median()
    df['Yas'] = df['Yas'].fillna(median_age)
    # Yaşı 15'ten küçük veya 100'den büyük olanları medyan yaş ile dolduralım
    df.loc[(df['Yas'] < 15) | (df['Yas'] > 100), 'Yas'] = median_age

# Üniversite okuma sayısını hesaplama
print(" - Üniversite okuma sayısı hesaplanıyor")
def calculate_university_count(row):
    if pd.isna(row['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']):
        return 1  # Sadece şu anki üniversite var
    else:
        return 2  # Hem şu anki üniversite hem de önceki var

# Yeni sütunu ekleyelim
train_df['Universite Okuma Sayisi'] = train_df.apply(calculate_university_count, axis=1)
test_df['Universite Okuma Sayisi'] = test_df.apply(calculate_university_count, axis=1)

# b. Gereksiz sütunların çıkarılması
print(" - Gereksiz sütunlar çıkarılıyor")
columns_to_drop = ['Dogum Tarihi', 'Lise Adi', 'Lise Adi Diger', 'Lise Bolum Diger', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite', 'Basvuru Tarihi']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# c. Binary sütunların kodlanması
print(" - Binary sütunlar kodlanıyor")
binary_columns = [
    'Burs Aliyor mu?', 'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Baska Bir Kurumdan Burs Aliyor mu?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?', 'Aktif olarak bir STK üyesi misiniz?',
    'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?', 'Ingilizce Biliyor musunuz?'
]

for col in binary_columns:
    for df in [train_df, test_df]:
        df[col] = df[col].astype(str).str.strip().str.lower()
        # Mapping sözlüğünü oluşturun
        mapping_dict = {'evet': 1, 'hayır': 0}
        # Değerleri kodlayalım
        df[col] = df[col].map(mapping_dict)
        # Eksik veya bilinmeyen değerleri 0 ile dolduralım
        df[col] = df[col].fillna(0)

# 'Cinsiyet' sütunu için kodlama
print(" - Cinsiyet sütunu kodlanıyor")
for df in [train_df, test_df]:
    df['Cinsiyet'] = df['Cinsiyet'].astype(str).str.strip().str.lower()
    gender_mapping = {'erkek': 'erkek', 'kadın': 'kadın', 'belirtmek istemiyorum': 'belirtmek istemiyorum'}
    df['Cinsiyet'] = df['Cinsiyet'].map(gender_mapping)
    df['Cinsiyet'] = df['Cinsiyet'].fillna('bilinmiyor')

# 'Universite Turu' sütunu için kodlama
print(" - Üniversite Türü sütunu kodlanıyor")
for df in [train_df, test_df]:
    df['Universite Turu'] = df['Universite Turu'].astype(str).str.strip().str.lower()
    uni_type_mapping = {'özel': 'özel', 'devlet': 'devlet'}
    df['Universite Turu'] = df['Universite Turu'].map(uni_type_mapping)
    df['Universite Turu'] = df['Universite Turu'].fillna('bilinmiyor')

# d. Eksik değerlerin doldurulması
print(" - Eksik değerler dolduruluyor")
# Kategorik sütunları belirleme
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()

# Sayısal ve kategorik sütunları ayırma
numerical_columns = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'Degerlendirme Puani' in numerical_columns:
    numerical_columns.remove('Degerlendirme Puani')  # Hedef değişkeni çıkardık

# Eğitim verisindeki eksik değerleri dolduralım
for df in [train_df, test_df]:
    for col in numerical_columns:
        median = train_df[col].median()
        df[col].fillna(median, inplace=True)
    for col in categorical_columns:
        mode = train_df[col].mode()
        if not mode.empty:
            df[col] = df[col].fillna(mode[0])
        else:
            df[col] = df[col].fillna('Bilinmiyor')

# e. Not Ortalamalarının sayısal değere dönüştürülmesi
print(" - Not ortalamaları sayısal değerlere dönüştürülüyor")
gpa_mapping = {
    '3.50-3': 3.25,
    '3.00-2.50': 2.75,
    '2.50 ve altı': 2.25,
    '3.00 - 4.00': 3.50,
    '3.50 - 4.00': 3.75,
    '2.50 - 3.00': 2.75,
    '2.00 ve altı': 1.75,
    '4.00 - 3.00': 3.50,
    '4.00 - 3.50': 3.75,
    '2.50-2': 2.25,
    '0': 0,
    'nan': np.nan,
    np.nan: np.nan
}

for df in [train_df, test_df]:
    df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].astype(str).str.strip()
    df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].map(gpa_mapping)
    df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].fillna(train_df['Universite Not Ortalamasi'].median())

# Benzer şekilde 'Lise Mezuniyet Notu' için
lise_not_mapping = {
    '3.50-3': 3.25,
    '3.00 - 4.00': 3.50,
    '3.00-2.50': 2.75,
    '2.50-2': 2.25,
    '2.00 ve altı': 1.75,
    '0': 0,
    'nan': np.nan,
    np.nan: np.nan
}

for df in [train_df, test_df]:
    df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].astype(str).str.strip()
    df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].map(lise_not_mapping)
    df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].fillna(train_df['Lise Mezuniyet Notu'].median())

# Hedef değişkende eksik değerleri kontrol edelim ve kaldıralım
print(" - Hedef değişkende eksik değerler kontrol ediliyor")
initial_length = len(train_df)
train_df.dropna(subset=['Degerlendirme Puani'], inplace=True)
final_length = len(train_df)
print(f" - {initial_length - final_length} satır eksik hedef değişken nedeniyle kaldırıldı")

# 3. Özellik Mühendisliği
print("3. Özellik Mühendisliği")

# a. Yeni özellikler
print(" - Yeni özellikler oluşturuluyor")
# 1. Ebeveyn Eğitim Seviyesi
education_mapping = {
    'eğitim yok': 0,
    'ilkokul mezunu': 1,
    'ortaokul mezunu': 2,
    'lise': 3,
    'üniversite': 4,
    'yüksek lisans / doktora': 5,
    'nan': 0,
    np.nan: 0
}

for df in [train_df, test_df]:
    df['Anne Egitim Durumu'] = df['Anne Egitim Durumu'].astype(str).str.strip().str.lower()
    df['Baba Egitim Durumu'] = df['Baba Egitim Durumu'].astype(str).str.strip().str.lower()
    df['Anne Egitim Seviyesi'] = df['Anne Egitim Durumu'].map(education_mapping).fillna(0).astype(int).astype(str)
    df['Baba Egitim Seviyesi'] = df['Baba Egitim Durumu'].map(education_mapping).fillna(0).astype(int).astype(str)
    df['Ebeveyn Egitim Ort'] = ((df['Anne Egitim Seviyesi'].astype(int) + df['Baba Egitim Seviyesi'].astype(int)) / 2).astype(float)

# 2. Aile Geliri Skoru
print(" - Aile geliri skoru hesaplanıyor")
sector_mapping = {
    'kamu': 2,
    'özel sektör': 2,
    'emekli': 1,
    'ev hanımı': 0,
    'diğer': 1,
    'nan': 0,
    np.nan: 0,
    '0': 0
}

for df in [train_df, test_df]:
    df['Anne Calisma Durumu'] = df['Anne Calisma Durumu'].astype(str).str.strip().str.lower()
    df['Baba Calisma Durumu'] = df['Baba Calisma Durumu'].astype(str).str.strip().str.lower()
    df['Anne Calisiyor'] = df['Anne Calisma Durumu'].map({'evet': 'evet', 'hayır': 'hayır'}).fillna('bilinmiyor')
    df['Baba Calisiyor'] = df['Baba Calisma Durumu'].map({'evet': 'evet', 'hayır': 'hayır'}).fillna('bilinmiyor')
    df['Anne Sektor'] = df['Anne Sektor'].astype(str).str.strip().str.lower().fillna('bilinmiyor')
    df['Baba Sektor'] = df['Baba Sektor'].astype(str).str.strip().str.lower().fillna('bilinmiyor')
    df['Anne Sektor Skoru'] = df['Anne Sektor'].map(sector_mapping).fillna(0).astype(int)
    df['Baba Sektor Skoru'] = df['Baba Sektor'].map(sector_mapping).fillna(0).astype(int)
    df['Anne Gelir Skoru'] = df['Anne Egitim Seviyesi'].astype(int) * df['Anne Sektor Skoru']
    df['Baba Gelir Skoru'] = df['Baba Egitim Seviyesi'].astype(int) * df['Baba Sektor Skoru']
    df['Aile Geliri'] = df['Anne Gelir Skoru'] + df['Baba Gelir Skoru']

# 3. STK Deneyimi
print(" - STK deneyimi özelliği oluşturuluyor")
for df in [train_df, test_df]:
    df['STK Deneyimi'] = df['Aktif olarak bir STK üyesi misiniz?'] + df['Stk Projesine Katildiniz Mi?']

# 4. Metin Verilerinden Özellik Çıkarımı
print(" - Metin verilerinden özellik çıkarılıyor")
# "Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?" sütunundan kelime sayısını alalım
for df in [train_df, test_df]:
    df['Girisimcilik Deneyim Kelime Sayisi'] = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].astype(str).apply(lambda x: len(x.split()))

# 5. İngilizce Seviyesi
print(" - İngilizce seviyesi kodlanıyor")
english_level_mapping = {
    'ana dilim gibi': 5,
    'çok iyi': 4,
    'iyi': 3,
    'orta': 2,
    'başlangıç': 1,
    'nan': 0,
    np.nan: 0
}

for df in [train_df, test_df]:
    df['Ingilizce Seviyeniz?'] = df['Ingilizce Seviyeniz?'].astype(str).str.strip().str.lower()
    df['Ingilizce Seviyesi'] = df['Ingilizce Seviyeniz?'].map(english_level_mapping).fillna(0).astype(int)

# 4. Modelleme
print("4. Modelleme")

# a. Hedef değişkeni ve özellikleri ayırma
print(" - Hedef değişken ve özellikler ayrılıyor")
X = train_df.drop(['Degerlendirme Puani', 'id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)
y = train_df['Degerlendirme Puani']

test_features = test_df.drop(['id', 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'], axis=1)

# b. Kategorik değişkenlerin belirlenmesi
print(" - Kategorik değişkenler belirleniyor")
# CatBoost modeline kategorik değişkenleri belirtelim
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Ayrıca kategorik olup sayısal olarak kodlanmış sütunları ekleyelim
categorical_cols += [
    'Cinsiyet', 'Universite Turu', 'Universite Okuma Sayisi', 'Anne Calisiyor', 'Baba Calisiyor',
    'Anne Egitim Seviyesi', 'Baba Egitim Seviyesi'
]

# Kategorik değişkenleri string tipine dönüştürelim
for col in categorical_cols:
    X[col] = X[col].astype(str)
    test_features[col] = test_features[col].astype(str)

# c. Model Eğitimi ve Hiperparametre Optimizasyonu
print(" - Model eğitimi ve hiperparametre optimizasyonu başlıyor")

# CatBoost modelini ve hiperparametrelerini tanımlayalım
model = CatBoostRegressor(verbose=0, random_state=42, loss_function='RMSE')

params = {
    'iterations': [500, 1000, 1500],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.03, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7]
}

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

randomized_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=params,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_iter=20,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

randomized_search.fit(X, y, cat_features=categorical_cols)
best_rmse = np.sqrt(-randomized_search.best_score_)
print(f"En iyi RMSE: {best_rmse}")
print(f"En iyi hiperparametreler: {randomized_search.best_params_}\n")

# 5. En İyi Model ile Tahmin ve Sonuç Dosyasının Oluşturulması
print("5. En iyi model ile tahmin yapılıyor ve sonuç dosyası oluşturuluyor")

# En iyi modeli tüm eğitim verisi üzerinde eğitelim
print(" - En iyi model tüm veri üzerinde eğitiliyor")
best_model = randomized_search.best_estimator_
best_model.fit(X, y, cat_features=categorical_cols)

# Test verisi üzerinde tahmin yapalım
print(" - Test verisi üzerinde tahmin yapılıyor")
test_predictions = best_model.predict(test_features)

# Negatif tahminleri 0'a eşitleyelim
test_predictions = np.where(test_predictions < 0, 0, test_predictions)

submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})

submission.to_csv('submission.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")


1. Veri Setini Yükleme
2. Veri Ön İşleme ve Temizleme
 - Yaş hesaplanıyor
 - Üniversite okuma sayısı hesaplanıyor
 - Gereksiz sütunlar çıkarılıyor
 - Binary sütunlar kodlanıyor
 - Cinsiyet sütunu kodlanıyor
 - Üniversite Türü sütunu kodlanıyor
 - Eksik değerler dolduruluyor
 - Not ortalamaları sayısal değerlere dönüştürülüyor
 - Hedef değişkende eksik değerler kontrol ediliyor
 - 1 satır eksik hedef değişken nedeniyle kaldırıldı
3. Özellik Mühendisliği
 - Yeni özellikler oluşturuluyor
 - Aile geliri skoru hesaplanıyor
 - STK deneyimi özelliği oluşturuluyor
 - Metin verilerinden özellik çıkarılıyor
 - İngilizce seviyesi kodlanıyor
4. Modelleme
 - Hedef değişken ve özellikler ayrılıyor
 - Kategorik değişkenler belirleniyor
 - Model eğitimi ve hiperparametre optimizasyonu başlıyor
Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi RMSE: 5.618463804704658
En iyi hiperparametreler: {'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 1500, 'depth': 6}

5. En iyi model il